In [1]:
import tensorflow as tf
from keras import layers
from tensorflow.keras import applications 
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras import models

from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
from tqdm import tqdm

In [2]:
import numpy
import pandas

aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
images = []
labels = []

for idx in tqdm(range(len(aptos_df))):
#     images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png', 20))
    labels.append(aptos_df.iloc[idx]['diagnosis'])
#     images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#     labels.append(dr_df.iloc[idx]['level'])

from tensorflow.keras.utils import to_categorical
images = numpy.load('../input/aptosnumpy128/images.npy')
# images = numpy.array(images)
labels = numpy.array(labels)
# cat_labels = to_categorical(labels)

from sklearn.model_selection import train_test_split
images, x_test, labels, y_test = train_test_split(images, labels, 
                                                    test_size=0.3, shuffle=True,
                                                    random_state=42, stratify=labels)

100%|██████████| 3662/3662 [00:00<00:00, 8603.68it/s]


In [3]:
class Metrics(Callback):
    def __init__(self, xval, yval):
        super().__init__()
        self.xval = xval
        self.yval = yval
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val = self.xval
        y_val = self.yval
        
        y_pred = self.model.predict(X_val)
        y_pred = numpy.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return
    
kappa_metrics = Metrics(images, labels)

In [4]:
xception = applications.Xception(include_top=False, weights='imagenet')

83689472/83683744 [==============================] - 2s 0us/step


In [5]:
def build_model():
    model = Sequential()
    model.add(xception)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256))
    model.add(layers.Dense(256))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [6]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 21,452,073
Trainable params: 21,397,545
Non-trainable params: 54,528
_________________________________________________________________


In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
skf.get_n_splits(images, labels)

4

In [8]:
for train_index, test_index in skf.split(images, labels):
    x_train, x_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    history = model.fit(x_train, y_train, epochs=8, batch_size=32,
    validation_data = (x_test, y_test), callbacks=[kappa_metrics])

Epoch 1/8
61/61 [==============================] - 41s 545ms/step - loss: 1.2826 - accuracy: 0.4366 - val_loss: 0.9201 - val_accuracy: 0.3822
val_kappa: 0.4665
Validation Kappa has improved. Saving model.
Epoch 2/8
61/61 [==============================] - 32s 517ms/step - loss: 0.2954 - accuracy: 0.5443 - val_loss: 0.5507 - val_accuracy: 0.4119
val_kappa: 0.7791
Validation Kappa has improved. Saving model.
Epoch 3/8
61/61 [==============================] - 32s 517ms/step - loss: 0.1720 - accuracy: 0.5605 - val_loss: 0.4516 - val_accuracy: 0.5101
val_kappa: 0.9087
Validation Kappa has improved. Saving model.
Epoch 4/8
61/61 [==============================] - 32s 517ms/step - loss: 0.1108 - accuracy: 0.5779 - val_loss: 0.6277 - val_accuracy: 0.4914
val_kappa: 0.9522
Validation Kappa has improved. Saving model.
Epoch 5/8
61/61 [==============================] - 31s 517ms/step - loss: 0.1595 - accuracy: 0.5954 - val_loss: 0.4340 - val_accuracy: 0.5554
val_kappa: 0.8542
Epoch 6/8
61/61 [===

In [12]:
from keras.models import load_model
model = load_model('./model.h5')

In [13]:
def predict(X, coef):
    X_p = numpy.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        else:
            X_p[i] = 4
    return X_p

In [14]:
from sklearn.metrics import confusion_matrix
pred = predict(model.predict(x_test), [0.5, 1.5, 2.5, 3.5])
print(confusion_matrix(y_test, pred))

[[314   1   0   0   0]
 [  0  65   0   0   0]
 [  0   0 173   2   0]
 [  0   0   1  32   1]
 [  0   0   0   2  49]]
